評価準備

### インストール

* pip installは失敗することがあるので、失敗したらもう一度実行してください

* githubから必要なファイルをコピーしてきます。gitが使える環境が必須です。


In [1]:
%%capture
# 1. ライブラリのインストール
# 必要なライブラリをインストール
#%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -U torch
!pip install -U peft

!pip install openai
!pip install unsloth-zoo # Install or update unsloth-zoo package
!pip install --upgrade --no-cache-dir "unsloth-zoo @ git+https://github.com/unslothai/unsloth-zoo.git"

!pip install --upgrade openai
#!pip install --upgrade transformers[olmo2]

!pip install openai #==1.55.3 # This line installs openai==1.55.3
!pip install --upgrade transformers
!pip install --upgrade trl

In [2]:
# 推論＆評価用ライブラリ h.godai
!pip install --upgrade git+https://github.com/h-godai/llm

  Cloning https://github.com/h-godai/llm to /tmp/pip-req-build-4upt5l1r
  Running command git clone --filter=blob:none --quiet https://github.com/h-godai/llm /tmp/pip-req-build-4upt5l1r
  Resolved https://github.com/h-godai/llm to commit aecd827da1d045c7799096c7800f25a7def1b7a5
  Preparing metadata (setup.py) ... done


In [3]:
# 必要なファイルをgithubから持ってきます。環境構築した直後に一度だけ実行してください。

!git clone https://github.com/h-godai/llm.git godai_temp
!cp -rv "godai_temp/LLM2024_env/" .
!rm -r godai_temp

Cloning into 'godai_temp'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 35 (delta 9), reused 18 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (35/35), 79.07 KiB | 622.00 KiB/s, done.
Resolving deltas: 100% (9/9), done.
'godai_temp/LLM2024_env/LLM2024_Model_Inference_fewshot2_20241217-L4.ipynb' -> './LLM2024_env/LLM2024_Model_Inference_fewshot2_20241217-L4.ipynb'
'godai_temp/LLM2024_env/elyza-tasks-100-TV_0.jsonl' -> './LLM2024_env/elyza-tasks-100-TV_0.jsonl'
'godai_temp/LLM2024_env/fewshot_prompt.json' -> './LLM2024_env/fewshot_prompt.json'


２回目以降はここから実行してください。

In [4]:
from google.colab import userdata

# 各APIキーの取得
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
HF_TOKEN = userdata.get('HF_TOKEN')

# 必要なライブラリを読み込み
from unsloth import FastLanguageModel
from peft import PeftModel
import torch
import json
from tqdm import tqdm
import re

import inference_evaluator.InferenceEvaluator as FireStorm

DataDir = "./LLM2024_env"

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [5]:
# モデルのロード
from huggingface_hub import login
from peft import LoraConfig

adapter_id = None
dpo_adapter_id = None
model_id = "llm-jp/llm-jp-3-13b"
#model_id = "llm-jp-3-3.7b-finetune-8-10k-ep1-2e-4"
#model_id = "llm-jp-3-13b-finetune-8-1k_combined-25k_ep1_merged-df16"
#model_id = "llm-jp-3-13b-ft-8-combined2-33k_ep1_2e-04" # 2.52
#model_id = "h-godai/llm-jp-3-13b-ft-8-combined3-73k_ep1_2e-04"
#model_id = "llm-jp-3-13b-finetune-8-1k_combined-25k_ep1_merged"
#adapter_id = "llm-jp-3-13b-finetune-8-1k_combined-25k-2e-4_LoRA"
#adapter_id = "llm-jp-3-13b-ft-8-combined3-73k_ep1_5e-05_32_32_LoRA"
#adapter_id = "llm-jp-3-13b-ft-8-combined4-32k_ep1_2e-04_32_32_LoRA"
#adapter_id = "llm-jp-3-13b-finetune-8-1k_combined-25k_ep1_2e-4_LoRA"
#adapter_id = "llm-jp-3-13b-ft8-cm25k-dpo2_1e-04_ep1_x1_bad32-3.27"
#adapter_id = "llm-jp-3-13b-ft8-cm25k-dpo2_1e-04_ep1_x1_bad32-3.45"
#adapter_id = "llm-jp-3-13b-ft8-cm25k-dpo2_2e-04_ep1_x0_bad34-3.30"
#dpo_adapter_id = "llm-jp-3-13b-finetune-8-1k_combined-25k_ep1_merged-dpo1_LoRA"
#adapter_id = "h-godai/llm-jp-3-13b-ft9-64k-2e-4-dpo2_1e-04_ep1_x1_bad33-3.20_LoRA"
#adapter_id = "llm-jp-3-13b-finetune-8-1k_combined-25k_ep1_2e-4_LoRA"
#model_id = "llm-jp-3-13b-ft8-cm25k-dpo2_1e-04_ep1_x1_bad32-3.27-merged"
#adapter_id = "h-godai/llm-jp-3-13b-ft8-cm25k-dpo3_fs2_LoRA"                # Release 12-14
#adapter_id = "h-godai/llm-jp-3-13b-ft8-cm25k-dpo1.5x2_bad28_3.44_fs2_LoRA" # Release 15
#adapter_id = "h-godai/llm-jp-3-13b-ft8-cm25k-dpo1.5_1e-04_ep1_x2_bad28-3.44_LoRA"
#adapter_id = "h-godai/llm-jp-3-13b-ft8-cm25k-dpo2_1e-04_ep1_x1_bad32-3.27_LoRA" # Rrelease 13
adapter_id = "h-godai/llm-jp-3-13b-ft8-cm25k-dpo1.5x2_bad28_3.44_fs2_LoRA" # Release 15,19

model_name = model_id if adapter_id is None else adapter_id


# unslothのFastLanguageModelで元のモデルをロード。
dtype = None # Noneにしておけば自動で設定
load_in_4bit = True # 今回は13Bモデルを扱うためTrue

if adapter_id:
  # LoRAと元のモデルを一緒にロード
  model, tokenizer = FastLanguageModel.from_pretrained(
      model_name=adapter_id,
      dtype=dtype,
      load_in_4bit=load_in_4bit,
      trust_remote_code=True,
  )
else:
  # 元モデルのみロード
  model, tokenizer = FastLanguageModel.from_pretrained(
      model_name=model_id,
      dtype=dtype,
      load_in_4bit=load_in_4bit,
      trust_remote_code=True,
  )


# 別途DPOがある場合
if dpo_adapter_id:
  #if adapter_id:
  #  model = model.merge_and_unload() # いったんモデルを結合する
  # Load the adapter configuration, potentially ignoring unknown keys
  config = LoraConfig.from_pretrained(dpo_adapter_id, ignore_mismatched_sizes=True)
  model = PeftModel.from_pretrained(model, dpo_adapter_id, config=config)  # Use the loaded config
  print(f"DPO LoRA adapter loaded from {dpo_adapter_id}")

model = FastLanguageModel.for_inference(model) # This line is added to fix the error

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.47.0.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.9. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Unsloth 2024.12.4 patched 40 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [6]:
import json

class FewShotGenerator:

  def __init__(self, fewshot_prompt = None):
    self.fewshot_prompt = fewshot_prompt

  def save(self, path):
    with open(path, 'w', encoding='utf-8') as f:
      json.dump(self.fewshot_prompt, f, indent=2, ensure_ascii=False)

  def load(self, path):
    with open(path, 'r', encoding='utf-8') as f:
      self.fewshot_prompt = json.load(f)

  # [{ "keywords": [<keyword[:Group]>,...], "fewshot1": <fewshot>, "endshot": <tailshot> }]
  # Group指定がある場合、同一グループのいずれかがマッチすればOK
  # それ以外は全マッチが必要
  #
  def input_prompt_hook(self, eval, prompt):
    for fsp in self.fewshot_prompt:
      kwlen = len(fsp["keywords"])
      ok = True
      group = {}
      for keyword in fsp["keywords"]:
        if ':' in keyword:
          # group付、group内のいずれかでOK
          words = keyword.split(':')
          keyword = words[0]
          gr = words[1]
          hit = keyword in prompt
          if gr not in group:
            group[gr] = 0
          if hit:
            group[gr] += 1
        else:
          # groupなし。全て一致する必要あり
          if keyword not in prompt:
            ok = False; # 一つでもなければNG
          pass
        pass
      pass # for keyword in
      if ok:
        # グループに0があればNG
        for gr in group:
          if group[gr] == 0:
            ok = False;
            break
          pass
        pass
      pass
      if ok and fsp["fewshot1"] is not None:
        if 'endshot' in fsp:
          return f"{fsp['fewshot1']}\n### 指示\n{prompt}\n\n{fsp['endshot']}\n### 回答\n"
        else:
          return f"{fsp['fewshot1']}\n### 指示\n{prompt}\n\n### 回答\n"
    pass # for fsp in
    # ない場合はデフォルト
    return f"{eval.prefix_prompt_}\n### 指示\n{prompt}\n\n### 回答\n"
  pass # def
pass  # class

fsg = FewShotGenerator()
fsg.load(f"{DataDir}/fewshot_prompt.json")



ここら↓は、LLM2024提出用のJsonl出力


In [7]:
# データセットの読み込み。
# omnicampusの開発環境では、左にタスクのjsonlをドラッグアンドドロップしてから実行。
datasets = []
with open(f"{DataDir}/elyza-tasks-100-TV_0.jsonl", "r") as f:
    item = ""
    for line in f:
      line = line.strip()
      item += line
      if item.endswith("}"):
        datasets.append(json.loads(item))
        item = ""

In [8]:
# 推論開始

import inference_evaluator.InferenceEvaluator as FireStorm

FastLanguageModel.for_inference(model)

evaltask = FireStorm.InferenceEvaluator(model, None) # inference only
evaltask.tokenizer_ = tokenizer
evaltask.max_tokens_ = 1408 # 1024 # max_tokens
evaltask.temperature_ = 0.2 # temperature
evaltask.repetition_penalty_ = 1.2 # repetition_penalty
evaltask.do_sample_ = evaltask.temperature_ > 0
evaltask.top_p = 0.9 # top_p

evaltask.prefix_prompt_ = "" # "以下の指示に厳密に従って、正確に回答してください。\n"
evaltask.suffix_prompt_ = ""
evaltask.input_prompt_hook_ = fsg.input_prompt_hook # few shot hook

results = []
for data in datasets:
  print(f"TaskId: {evaltask.eval_count_}")
  input = data["input"]
  output = evaltask.inference(input)
  print(output)
  results.append({"task_id": data["task_id"], "input": input, "output": output})
  evaltask.eval_count_ += 1


not eval mode
TaskId: 0

======= TASK-0 :


### 指示
野球選手が今シーズン活躍するために取り組むべき5つのことを教えてください。

### 回答

野球選手が今シーズン活躍するための取り組みは以下の通りです。

1. 体力強化: シーズンを通してパフォーマンスを維持するため、筋力トレーニングや持久力向上のためのランニングに重点的に取り組んでください。また、怪我予防のためにストレッチも欠かさずに行ってください。
2. 技術練習: 野球技術の上達のため、素振りなどの基本的な動作から変化球への対応、守備・走塁といった応用まで幅広く練習してください。特に苦手な部分については繰り返し練習し、克服することが重要です。
3. メンタル面のケア: 試合でのプレッシャーや緊張と向き合うため、メンタルトレーニングで集中力アップやリラックス法などを学びましょう。また睡眠時間をしっかりと確保することで、心身ともにリフレッシュすることも大切です。
4. 食事管理: パフォーマンス向上のために、バランスの良い食事をとることが必要不可欠です。試合前のエネルギー補給や、疲労回復のための栄養摂取など、目的に応じた適切な食事内容を心がけることが重要です。
5. チームとのコミュニケーション: チームワークが必要となるスポーツである以上、チームメイトとの良好な関係構築は非常に大事なことです。積極的に声をかけたり、ミーティングに参加したりして、チーム全体としての一体感を持つように努めてみてください。
TaskId: 1

======= TASK-1 :


### 指示
花の都で公共ピアノが設置され、訪れた人々が自由にピアノを弾く様子が定点カメラで撮影されています。大学生が好きな歌を弾き、観光の若者が楽器を演奏するセッションが実現する一方、子育てを終えた母親がピアノを再開し、そのピアノに合わせて音楽コンクールで優勝した中学生が歌います。また、バーテンダーが駅ピアノを楽しむ姿や、男性が亡き友人の思い出を語りながらシャンソンを口ずさむ姿も見られます。このような情景を織り交ぜた短編小説を書いてください。

### 回答

花の都では公共ピアノが設置され、様々な人が集う光景が繰り広げられています。

大学生は大好きな曲を弾いて楽しそうに歌い上げます。観光の若者たち

In [9]:
# こちらで生成されたjsolを提出してください。
# 本コードではinputも含んでいますが、なくても問題ありません。
# 必須なのはtask_idとoutputとなります。
import re
import datetime
import pytz
now = datetime.datetime.now(pytz.timezone("Asia/Tokyo")).strftime('%Y%m%d_%H%M%S')
with open(f"./outputs-{now}.jsonl", 'w', encoding='utf-8') as f:
    for result in results:
      print(result)
      json.dump(result, f, ensure_ascii=False)  # ensure_ascii=False for handling non-ASCII characters
      f.write('\n')

{'task_id': 0, 'input': '野球選手が今シーズン活躍するために取り組むべき5つのことを教えてください。', 'output': '野球選手が今シーズン活躍するための取り組みは以下の通りです。\n\n1. 体力強化: シーズンを通してパフォーマンスを維持するため、筋力トレーニングや持久力向上のためのランニングに重点的に取り組んでください。また、怪我予防のためにストレッチも欠かさずに行ってください。\n2. 技術練習: 野球技術の上達のため、素振りなどの基本的な動作から変化球への対応、守備・走塁といった応用まで幅広く練習してください。特に苦手な部分については繰り返し練習し、克服することが重要です。\n3. メンタル面のケア: 試合でのプレッシャーや緊張と向き合うため、メンタルトレーニングで集中力アップやリラックス法などを学びましょう。また睡眠時間をしっかりと確保することで、心身ともにリフレッシュすることも大切です。\n4. 食事管理: パフォーマンス向上のために、バランスの良い食事をとることが必要不可欠です。試合前のエネルギー補給や、疲労回復のための栄養摂取など、目的に応じた適切な食事内容を心がけることが重要です。\n5. チームとのコミュニケーション: チームワークが必要となるスポーツである以上、チームメイトとの良好な関係構築は非常に大事なことです。積極的に声をかけたり、ミーティングに参加したりして、チーム全体としての一体感を持つように努めてみてください。'}
{'task_id': 1, 'input': '花の都で公共ピアノが設置され、訪れた人々が自由にピアノを弾く様子が定点カメラで撮影されています。大学生が好きな歌を弾き、観光の若者が楽器を演奏するセッションが実現する一方、子育てを終えた母親がピアノを再開し、そのピアノに合わせて音楽コンクールで優勝した中学生が歌います。また、バーテンダーが駅ピアノを楽しむ姿や、男性が亡き友人の思い出を語りながらシャンソンを口ずさむ姿も見られます。このような情景を織り交ぜた短編小説を書いてください。', 'output': '花の都では公共ピアノが設置され、様々な人が集う光景が繰り広げられています。\n\n大学生は大好きな曲を弾いて楽しそうに歌い上げます。観光の若者たちは即興でセッションし合い、その場限りのハ

ELYZA-tasks-100の評価を行います。
デフォルトではGeminiのAPIを使います。

In [10]:
# ELYZA-task-100の評価を行う

import inference_evaluator.InferenceEvaluator as FireStorm
from datasets import load_dataset

if False:
  # ELYZA-tasks-100をダウンロードしてデータセットとして読み込み
  elyza100_datasets = load_dataset("elyza/ELYZA-tasks-100")
  !mkdir outputs

  # 評価実行
  #evaltask = InferenceEvaluator(model, OPENAI_API_KEY)
  evaltask = FireStorm.InferenceEvaluator(model, GEMINI_API_KEY, "gemini") # using GEMINI
  evaltask.prefix_prompt_ = "" #"以下の指示に厳密に従って、正確に回答してください。\n"
  #evaltask.suffix_prompt_ = "" # "簡潔に回答してください\n"
  evaltask.input_prompt_hook_ = fsg.input_prompt_hook # few shot hook
  evaltask.run(dataset = elyza100_datasets['test'],
              name = model_name+"_fs2",
              tokenizer = tokenizer,
              temperature = 0.0,
              repetition_penalty = 1.2, #1.2,
              max_tokens= 1024 # 1408 # 1024 # 1408 # 1024 #1280
  )
  evaltask.write_result()
